# TESTE AUTOENCODER

In [6]:
# Cell for pip installing required packages on separate lines


In [7]:
# Cell 1: Importações e configurações iniciais
import os
import cv2
import mediapipe as mp
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

import numpy as np
import joblib

from keras.models import load_model
import matplotlib.pyplot as plt
#from keras import layers

In [8]:
# Cell 2
# Configuração do MediaPipe Holistic
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Funções auxiliares
def angle_between_points(p1, p2, p3):
    a = np.array(p1)
    b = np.array(p2)
    c = np.array(p3)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle

def get_coords(landmark, height, width):
    return int(landmark.x * width), int(landmark.y * height)

# Carregar o modelo LSTM treinado
model = load_model('InfinityRepVideos/armraise/model/221223_100_50epochs_16batch_dropout02_autoencoder_lstm_best_model.h5')

# Carregar o scaler salvo
scaler = joblib.load('InfinityRepVideos/armraise/model/autoencoder_lstm_scaler.save')

# Função para preparar os dados para previsão
def prepare_data_for_prediction(data, n_steps, window_size=3):
    X = []
    for i in range(len(data) - n_steps):
        seq_x = data[i:i + n_steps]
        ma_seq_x = np.array([moving_average(seq, window_size) for seq in seq_x.T]).T  # Aplicar média móvel
        X.append(ma_seq_x)
    return np.array(X)

def moving_average(data, window_size):
    # Substituir NaNs por zeros, aplicar convolução, e substituir zeros de volta por NaNs
    data_nan_mask = np.isnan(data)
    data_filled = np.where(data_nan_mask, 0, data)
    ma = np.convolve(data_filled, np.ones(window_size)/window_size, mode='same')
    ma[data_nan_mask] = np.nan  # Substituir de volta os NaNs originais
    return ma

# Função para calcular a perda de reconstrução
def calculate_reconstruction_loss(original, reconstructed):
    return np.mean(np.square(original - reconstructed))

In [9]:
# Cell 3: Avaliação do vídeo

n_steps = 48
frames = []
frame_count = 0
sequence_evaluation_results = []  # To store the evaluation results of each sequence

# Carregar vídeo pré-gravado
cap = cv2.VideoCapture('InfinityRepVideos/wrong/video_2023-11-24_17-48-53.mp4')
reconstructed_frames = []

# Inicialize reconstructed_sequence_inversed como None
reconstructed_sequence_inversed = None

# Limiar de detecção de anomalias
threshold = 0.01888204895653068

while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        break

    frame_count += 1
    frame = cv2.resize(frame, (frame.shape[1]//2, frame.shape[0]//2))
    results = holistic.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Desenhar o esqueleto do MediaPipe
    mp.solutions.drawing_utils.draw_landmarks(frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

    if results.pose_landmarks:
        height, width, _ = frame.shape
        angles = {}

        angles["angle_right_elbow"] = angle_between_points(
            get_coords(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER.value], height, width),
            get_coords(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW.value], height, width),
            get_coords(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST.value], height, width))
        angles["angle_left_elbow"] = angle_between_points(
            get_coords(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER.value], height, width),
            get_coords(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW.value], height, width),
            get_coords(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST.value], height, width))

        angles["angle_right_shoulder"] = angle_between_points(
            get_coords(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW.value], height, width),
            get_coords(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER.value], height, width),
            get_coords(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER.value], height, width))

        angles["angle_left_shoulder"] = angle_between_points(
            get_coords(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW.value], height, width),
            get_coords(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER.value], height, width),
            get_coords(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER.value], height, width))

        angles["angle_right_knee"] = angle_between_points(
            get_coords(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP.value], height, width),
            get_coords(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_KNEE.value], height, width),
            get_coords(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ANKLE.value], height, width))

        angles["angle_left_knee"] = angle_between_points(
            get_coords(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP.value], height, width),
            get_coords(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_KNEE.value], height, width),
            get_coords(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ANKLE.value], height, width))

        angles["angle_right_hip"] = angle_between_points(
            get_coords(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_KNEE.value], height, width),
            get_coords(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP.value], height, width),
            get_coords(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP.value], height, width))

        angles["angle_left_hip"] = angle_between_points(
            get_coords(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_KNEE.value], height, width),
            get_coords(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP.value], height, width),
            get_coords(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP.value], height, width))
        
        angles["angle_right axilla"] = angle_between_points(
            get_coords(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER.value], height, width),
            get_coords(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP.value], height, width),
            get_coords(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW.value], height, width))
        
        angles["angle_left axilla"] = angle_between_points(
            get_coords(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER.value], height, width),
            get_coords(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP.value], height, width),
            get_coords(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW.value], height, width))
        
        angles["angle_right thigh"] = angle_between_points(
            get_coords(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP.value], height, width),
            get_coords(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP.value], height, width),
            get_coords(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_KNEE.value], height, width))
        
        angles["angle_left thigh"] = angle_between_points(
            get_coords(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP.value], height, width),
            get_coords(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP.value], height, width),
            get_coords(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_KNEE.value], height, width))
        
        # Converter o dicionário de ângulos para uma lista e adicionar a frames
        frames.append(list(angles.values()))

        if len(frames) == n_steps:
            sequence_data = np.array(frames)
            sequence_data_scaled = scaler.transform(sequence_data)
            sequence_data_reshaped = sequence_data_scaled.reshape(1, n_steps, -1)

            reconstructed_sequence = model.predict(sequence_data_reshaped)
            reconstruction_loss = calculate_reconstruction_loss(sequence_data_scaled, reconstructed_sequence[0])

            sequence_evaluation_results.append(reconstruction_loss)

            # Limpar a lista de quadros para o próximo segmento
            frames.clear()

    # Display frame count and current segment number (top-left corner)
    current_segment = frame_count // n_steps
    cv2.putText(frame, f'Frame: {frame_count} (Seg: {current_segment + 1})', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    # Display evaluation result of the last sequence and its segment number (below frame count)
    if sequence_evaluation_results:
        last_loss = sequence_evaluation_results[-1]
        eval_segment = len(sequence_evaluation_results)
        eval_text = f'Seg {eval_segment}: {"Correct" if last_loss < threshold else "Incorrect"} (Loss: {last_loss:.4f})'
        cv2.putText(frame, eval_text, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0) if last_loss < threshold else (0, 0, 255), 2, cv2.LINE_AA)

    # Display captured angles (bottom-left corner)
    y_offset = frame.shape[0] - 200
    for angle_name, angle_value in angles.items():
        cv2.putText(frame, f'{angle_name}: {angle_value:.2f}', (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        y_offset += 20

    cv2.imshow('Frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 39ms/step


In [10]:
# # Cell 4: Plotar gráficos
# # Converter as listas de reconstruções e valores reais para arrays numpy
# reconstructed = np.array(reconstructed_frames)
# real = np.array(frames)

# # Certifique-se de que os arrays têm o mesmo tamanho
# min_length = min(len(reconstructed), len(real))
# reconstructed_aligned = reconstructed[:min_length]
# real_aligned = real[-min_length:]

# # Nomes dos ângulos para a legenda dos gráficos
# angle_names = [
#     "angle_right_elbow", "angle_left_elbow",
#     "angle_right_shoulder", "angle_left_shoulder",
#     "angle_right_knee", "angle_left_knee",
#     "angle_right_hip", "angle_left_hip",
#     "angle_right_axilla", "angle_left_axilla",
#     "angle_right_thigh", "angle_left_thigh",
# ]

# # Plotar gráficos
# plt.figure(figsize=(15, 20))

# for i in range(12): 
#     plt.subplot(6, 2, i + 1)
#     plt.plot(reconstructed_aligned[:, i], label='Reconstruído')
#     plt.plot(real_aligned[:, i], label='Original')
#     plt.title(angle_names[i])
#     plt.legend()

# plt.tight_layout()
# plt.show()